<a href="https://colab.research.google.com/github/Phudit-2547/Chunimai_spend_tracker/blob/main/chunimai_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install playwright
!python3 -m playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 15.8 MB/s eta 0:00:00
163.5 MiB [] 0% 10.8s163.5 MiB [] 0% 42.7s163.5 MiB [] 0% 21.6s163.5 MiB [] 0% 13.2s163.5 MiB [] 0% 9.7s163.5 MiB [] 1% 6.0s163.5 MiB [] 2% 4.3s163.5 MiB [] 2% 4.2s163.5 MiB [] 3% 3.9s163.5 MiB [] 3% 3.8s163.5 MiB [] 4% 3.6s163.5 MiB [] 5% 3.5s163.5 MiB [] 5% 3.4s163.5 MiB [] 6% 3.6s163.5 MiB [] 7% 3.7s163.5 MiB [] 8% 3.4s163.5 MiB [] 8% 3.5s163.5 MiB [] 9% 3.3s163.5 MiB [] 9% 3.2s163.5 MiB [] 10% 3.2s163.5 MiB [] 10% 3.1s163.5 MiB [] 11% 3.1s163.5 MiB [] 12% 3.0s163.5 MiB [] 13% 2.9s163.5 MiB [] 14% 2.7s163.5 MiB [] 15% 2.7s163.5 MiB [] 16% 2.6s163.5 MiB [] 17% 2.5s163.5 MiB [] 18% 2.3s163.5 MiB [] 19% 2.3s163.5 MiB [] 21% 2.1s163.5 MiB [] 22% 2.0s163.5 MiB [] 23% 2.1s163.5 MiB [] 23% 2.0s163.5 MiB [] 24% 2.0s163.5 MiB [] 25% 2.0s163.5 MiB [] 26% 1.9s163.5 MiB [] 27% 1.9s163.5 MiB [] 28% 1.9s163.5 MiB [] 29% 1.8s163.5 MiB [] 30% 1.8s163.5 MiB [] 30% 1.7s163.5 MiB [] 31% 1.7s163.5 MiB [] 32% 1.7s163.5 MiB []

In [70]:
import asyncio
import json
import requests
from playwright.async_api import async_playwright
from google.colab import userdata
from datetime import datetime

# ✅ CONFIG: Enable or disable games
CONFIG = {
    "chunithm": True,  # Set to True if you want to scrape Chunithm
    "maimai": True     # Set to True if you want to scrape Maimai
}

DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1347056535976546447/-dmpYuiS8wGVdHng5BGmzfvmj6O_y6AOGApJXiM9pOJnwZfigz6JVC2WhgQkTFAbp0DJ"

async def login_and_get_play_count(game):
    """Logs into the game website and counts today's plays."""
    LOGIN_URLS = {
        "chunithm": "https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=chuniex&redirect_url=https://chunithm-net-eng.com/mobile/&back_url=https://chunithm.sega.com/",
        "maimai": "https://lng-tgk-aime-gw.am-all.net/common_auth/login?site_id=maimaidxex&redirect_url=https://maimaidx-eng.com/maimai-mobile/&back_url=https://maimai.sega.com/"
    }
    HOME_URLS = {
        "chunithm": "https://chunithm-net-eng.com/mobile/home/",
        "maimai": "https://maimaidx-eng.com/maimai-mobile/home/"
    }

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        print(f"Navigating to {game} login page...")
        await page.goto(LOGIN_URLS[game], wait_until="domcontentloaded")

        print("Waiting for 'SEGA ID' button...")
        sega_id_button = page.locator("span.c-button--openid--segaId")
        await sega_id_button.wait_for(state="visible", timeout=60000)
        await sega_id_button.click()

        print("Filling in login details...")
        await page.locator("#sid").fill(userdata.get('USERNAME'))
        await page.locator("#password").fill(userdata.get('PASSWORD'))

        print("Clicking 'Login' button...")
        await page.locator("input#btnSubmit.c-button--login").click()

        print(f"Waiting for {game} home page to load...")
        await page.wait_for_url(HOME_URLS[game])

        play_count = 0
        today_str = '2025/03/04' #datetime.today().strftime("%Y/%m/%d")

        if game == "chunithm":
            print("Navigating to Chunithm Record menu...")
            await page.goto("https://chunithm-net-eng.com/mobile/record/", wait_until="domcontentloaded")

            print("Clicking 'PLAY RECORD'...")
            play_record_button = page.locator("li.submenu_play a")
            await play_record_button.wait_for(state="visible", timeout=60000)
            await play_record_button.click()
            await page.wait_for_url("https://chunithm-net-eng.com/mobile/record/playlog")

            date_track_pairs = await page.locator("div.frame02.w400").evaluate_all(
                "elements => elements.map(e => [e.querySelector('.play_datalist_date')?.innerText.split(' ')[0], e.querySelector('.play_track_text')?.innerText.split(' ')[1]])"
            )

            checker = [track for date, track in date_track_pairs if date == today_str]
            if not checker:
                print("✅ No plays found for today.")
                return 0

            play_count = 1
            for i in range(len(checker) - 1):
                if int(checker[i]) < int(checker[i + 1]):
                    play_count += 1

        elif game == "maimai":
            print("Navigating to Maimai Record menu...")
            await page.goto("https://maimaidx-eng.com/maimai-mobile/record/", wait_until="domcontentloaded")

            records = await page.locator(f"text={today_str}").all()
            if not records:
                print("✅ No plays found for today.")
                return 0

            track_elements = await page.locator("div.sub_title.t_c.f_r.f_11").all_text_contents()
            checker = [txt.split()[1] for txt in track_elements if txt.split()[2] == today_str]

            play_count = 1
            for i in range(len(checker) - 1):
                if int(checker[i]) < int(checker[i + 1]):
                    play_count += 1

        print(f"✅ {game.capitalize()} plays today: {play_count}")
        await browser.close()
        return play_count


def send_discord_notification(game, play_count):
    """Sends a message to Discord only if play_count > 0."""
    if play_count > 0:
        message = {
            "username": "Game Scraper Bot",
            "content": f"🎵 **{game.capitalize()}**: You have played **{play_count}** credits today!"
        }
        response = requests.post(DISCORD_WEBHOOK_URL, data=json.dumps(message), headers={"Content-Type": "application/json"})

        if response.status_code == 204:
            print(f"✅ Sent Discord notification for {game}.")
        else:
            print(f"❌ Failed to send Discord message. Response: {response.text}")


async def main():
    if CONFIG["chunithm"]:
        chunithm_plays = await login_and_get_play_count("chunithm")
        send_discord_notification("chunithm", chunithm_plays)

    if CONFIG["maimai"]:
        maimai_plays = await login_and_get_play_count("maimai")
        send_discord_notification("maimai", maimai_plays)

# Run the function
await main()


Navigating to chunithm login page...
Waiting for 'SEGA ID' button...
Filling in login details...
Clicking 'Login' button...
Waiting for chunithm home page to load...
Navigating to Chunithm Record menu...
Clicking 'PLAY RECORD'...
✅ Chunithm plays today: 1
✅ Sent Discord notification for chunithm.
Navigating to maimai login page...
Waiting for 'SEGA ID' button...
Filling in login details...
Clicking 'Login' button...
Waiting for maimai home page to load...
Navigating to Maimai Record menu...
✅ Maimai plays today: 2
✅ Sent Discord notification for maimai.
